In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

# Acquire

In [2]:
df = pd.read_csv('diabetes_prediction_dataset.csv')

In [3]:
df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


# Prep

### Are there any missing values?

In [4]:
df.isna().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

- zero nulls

### Are there any duplicates?

In [5]:
df.duplicated(keep='first').sum()

3854

- 3854 duplicates 
- By dropping duplicate rows, we can ensure that the model is trained on a representative and unbiased dataset, which can lead to more accurate predictions and better performance overall.


In [6]:
df.drop_duplicates(ignore_index=True, inplace=True)

In [7]:
df.duplicated().sum()

0

### What are the data types?

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96146 entries, 0 to 96145
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               96146 non-null  object 
 1   age                  96146 non-null  float64
 2   hypertension         96146 non-null  int64  
 3   heart_disease        96146 non-null  int64  
 4   smoking_history      96146 non-null  object 
 5   bmi                  96146 non-null  float64
 6   HbA1c_level          96146 non-null  float64
 7   blood_glucose_level  96146 non-null  int64  
 8   diabetes             96146 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.6+ MB


### Shape?

In [9]:
df.shape

(96146, 9)

### Changing age to an int and not a float?

In [10]:
df.age.value_counts()

80.00    4932
51.00    1566
47.00    1511
48.00    1508
53.00    1498
         ... 
0.48       83
1.00       83
0.40       66
0.16       59
0.08       36
Name: age, Length: 102, dtype: int64

In [11]:
df[(df.age < 1) & (df.diabetes == 1)]

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes


In [12]:
df = df.drop(df[df.age < 1].index)


In [13]:
df.shape

(95236, 9)

- dropped all rows with the age less than 1 years old. 
- no ages below 1 have diabetes so it felt unneeded to keep.
- research also shows that it is rare for a child under the age of 1 to have diabetes although not impossible.
- for the sake of my data types I'm dropping them to be able to convert my age to an int

In [14]:
df.age = df.age.astype(int)

In [15]:
df.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80,0,1,never,25.19,6.6,140,0
1,Female,54,0,0,No Info,27.32,6.6,80,0
2,Male,28,0,0,never,27.32,5.7,158,0
3,Female,36,0,0,current,23.45,5.0,155,0
4,Male,76,1,1,current,20.14,4.8,155,0


### What are my value counts for the target now?

In [16]:
df.diabetes.value_counts()

0    86754
1     8482
Name: diabetes, dtype: int64

### Changing target name 'diabetes' to 'diabetic'

In [17]:
df = df.rename(columns={'diabetes': 'diabetic'})

In [18]:
df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetic
0,Female,80,0,1,never,25.19,6.6,140,0
1,Female,54,0,0,No Info,27.32,6.6,80,0
2,Male,28,0,0,never,27.32,5.7,158,0
3,Female,36,0,0,current,23.45,5.0,155,0
4,Male,76,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
96141,Female,36,0,0,No Info,24.60,4.8,145,0
96142,Female,2,0,0,No Info,17.37,6.5,100,0
96143,Male,66,0,0,former,27.83,5.7,155,0
96144,Female,24,0,0,never,35.42,4.0,100,0


In [19]:
df.gender.value_counts()

Female    55728
Male      39490
Other        18
Name: gender, dtype: int64

In [27]:
mask = df.gender == 'Other'

In [21]:
# df = df[df.gender == 'Other']

In [28]:
df[mask]

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetic
12589,Other,10,0,0,not current,14.09,5.0,140,0
14730,Other,19,0,0,No Info,27.32,5.7,158,0
16574,Other,39,0,0,not current,31.24,6.2,85,0
18540,Other,10,0,0,not current,16.59,6.1,160,0
23023,Other,23,0,0,No Info,24.23,6.1,140,0
31547,Other,53,0,0,No Info,27.32,6.6,160,0
33299,Other,45,0,0,never,27.32,4.0,159,0
34393,Other,47,0,0,never,36.76,6.6,90,0
34466,Other,47,0,0,never,36.76,3.5,200,0
39626,Other,18,0,0,not current,30.19,6.1,90,0


In [29]:
df = df.drop(df[mask].index)

In [31]:
df.columns


Index(['gender', 'age', 'hypertension', 'heart_disease', 'smoking_history',
       'bmi', 'HbA1c_level', 'blood_glucose_level', 'diabetic'],
      dtype='object')